In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

%cd ../..

!hostname

/p/fastdata/pli/Private/oberstrass1/datasets/vervet1818-3d
jrlogin03.jureca


In [3]:
import os

import re
import pandas as pd
import numpy as np

import pli
import pli.image as im

from tqdm import tqdm

In [4]:
# Load section infos

trans_path = "data/aa/ntransmittance/"
dir_path = "data/aa/direction/"
ret_path = "data/aa/retardation/"
mask_path = "data/aa/masks/cortex/"

# Group of the mask in the H5 files
mask_group = 'Image'

# Coressponding pyramid
mask_pyramid = 6
pli_pyramid = 2

###

import h5py as h5

p = re.compile('.*s([0-9]{4})_.*')

feature_list = []
for tf, df, rf in zip(sorted(os.listdir(trans_path)), sorted(os.listdir(dir_path)), sorted(os.listdir(ret_path))):
    id = int(p.match(tf)[1])
    id_2 = int(p.match(rf)[1])
    assert id == id_2
    feature_list.append({'id': id, 'file_trans': os.path.join(trans_path, tf),
                         'file_dir': os.path.join(dir_path, df),
                         'file_ret': os.path.join(ret_path, rf)})
feature_df = pd.DataFrame(feature_list).sort_values('id').reset_index(drop=True)

mask_list = []
for f in sorted(os.listdir(mask_path)):
    id = int(p.match(f)[1])
    mask_section = pli.data.Section(os.path.join(mask_path, f))
    spacing = tuple((2 ** mask_pyramid) * s for s in mask_section.spacing)
    origin = mask_section.origin
    mask_section.close_file_handle()
    mask_list.append({'id': id, 'spacing': spacing, 'origin': origin, 'file_mask': os.path.join(mask_path, f)})
mask_df = pd.DataFrame(mask_list).sort_values('id').reset_index(drop=True)

files_df = mask_df.merge(feature_df, on='id', how='left')

files_df.head()

,id,spacing,origin,file_mask,file_trans,file_dir,file_ret
0,841,"(84.37760192, 84.37760192)","[0.0, 0.0]",data/aa/masks/cortex/Vervet1818aa_60mu_70ms_s0...,data/aa/ntransmittance/Vervet1818aa_60mu_70ms_...,data/aa/direction/Vervet1818aa_60mu_70ms_s0841...,data/aa/retardation/Vervet1818aa_60mu_70ms_s08...
1,842,"(84.37760192, 84.37760192)","[0.0, 0.0]",data/aa/masks/cortex/Vervet1818aa_60mu_70ms_s0...,data/aa/ntransmittance/Vervet1818aa_60mu_70ms_...,data/aa/direction/Vervet1818aa_60mu_70ms_s0842...,data/aa/retardation/Vervet1818aa_60mu_70ms_s08...
2,843,"(84.37760192, 84.37760192)","[0.0, 0.0]",data/aa/masks/cortex/Vervet1818aa_60mu_70ms_s0...,data/aa/ntransmittance/Vervet1818aa_60mu_70ms_...,data/aa/direction/Vervet1818aa_60mu_70ms_s0843...,data/aa/retardation/Vervet1818aa_60mu_70ms_s08...
3,844,"(84.37760192, 84.37760192)","[0.0, 0.0]",data/aa/masks/cortex/Vervet1818aa_60mu_70ms_s0...,data/aa/ntransmittance/Vervet1818aa_60mu_70ms_...,data/aa/direction/Vervet1818aa_60mu_70ms_s0844...,data/aa/retardation/Vervet1818aa_60mu_70ms_s08...
4,845,"(84.37760192, 84.37760192)","[0.0, 0.0]",data/aa/masks/cortex/Vervet1818aa_60mu_70ms_s0...,data/aa/ntransmittance/Vervet1818aa_60mu_70ms_...,data/aa/direction/Vervet1818aa_60mu_70ms_s0845...,data/aa/retardation/Vervet1818aa_60mu_70ms_s08...


In [ ]:
# Fit PCA by subsample of the data

# Number of randomly selected sections
n_subsamples = 1 # 32  # len(files_df)

# Number of bins for pli features
n_bins = 16

seed = 299792458

# Masking of features to include foreground only
mask_features = True
background_class = 3

###

from vervet1818_3d.utils.io import pli2feat

np.random.seed(seed)

selected_features = []
selected_masks = []

section_ids = []

for k, r in tqdm(files_df.sample(n_subsamples).sort_values('id').iterrows(), total=n_subsamples):
    trans_section = pli.data.Section(r.file_trans)
    dir_section = pli.data.Section(r.file_dir)
    ret_section = pli.data.Section(r.file_ret)

    features = pli2feat(
        trans_section.pyramid[pli_pyramid][:],
        np.deg2rad(dir_section.pyramid[pli_pyramid]),
        ret_section.pyramid[pli_pyramid][:],
        mask_pyramid - pli_pyramid,
        n_bins
    )

    mask_section = pli.data.Section(r.file_mask, data_group=mask_group)
    mask = mask_section.pyramid[mask_pyramid][:]

    selected_features.append(features)
    selected_masks.append(mask)
    section_ids.append(r.id)

print(f"Use sections {section_ids}")

if mask_features:
    valid_features = [f[m != background_class] for f, m in zip(selected_features, selected_masks)]
else:
    valid_features = [sf.reshape(-1, sf.shape[-1]) for sf in selected_features]

valid_lengths = [len(vf) for vf in valid_features]
valid_features = np.vstack(valid_features)
del selected_features

print(f"Valid features have shape {valid_features.shape}")

  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

starting 8 processes to create features


In [ ]:
# Perform PCA

n_samples = 1_000_000

seed = 299792458

###

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

np.random.seed(seed)

ix = np.random.choice(np.arange(len(valid_features)), n_samples)
pca_components = len(valid_features[0])

feature_selection = valid_features[ix].astype(np.float64)

scaler = StandardScaler()
std_features = scaler.fit_transform(feature_selection)

pca = PCA(n_components=pca_components, whiten=False)
pca.fit(std_features)

In [ ]:
# Get optimal number of components

import matplotlib.pyplot as plt

from vervet1818_3d.utils.stats import profile_likelihood

pl = np.array([profile_likelihood(l, pca) for l in range(pca.n_components)])

pl_components = np.argmax(pl)
print(f"Optimum found at {pl_components} components")

plt.vlines(np.argmax(pl), pl.max(), pl.min(), colors='black', linestyles='--')
plt.plot(pl[:100]);

In [ ]:
# Function of variance explained

min_variance_explained = 0.7

###

import matplotlib.pyplot as plt

scree = np.cumsum(pca.explained_variance_)
scree /= scree[-1]

variance_components = np.argwhere(scree > min_variance_explained)[0, 0] + 1

n_components = max(pl_components, variance_components)

variance_explained = scree[n_components -1] / scree[-1]

print(f"Explained variance: {100 * variance_explained:.2f}% with {n_components} components")
plt.plot(scree[:100])
plt.vlines(n_components, scree.max(), 0, colors='black', linestyles='--')
plt.title("Proportion of variance explained")
plt.xlabel("#Components")
plt.grid()
plt.show()

In [20]:
out_folder = "data/aa/pca/"
pca_key = "PCA"
valid_key = "Valid"

###

from vervet1818_3d.utils.stats import pca_n_transform

import h5py as h5

out_path = os.path.join(out_folder, 'pli')

if not os.path.exists(out_path):
    os.mkdir(out_path)

for k, r in tqdm(files_df.iterrows(), total=len(files_df)):
    trans_section = pli.data.Section(r.file_trans)
    dir_section = pli.data.Section(r.file_dir)
    ret_section = pli.data.Section(r.file_ret)

    test_features = pli2feat(trans_section.pyramid[pli_pyramid][:],
                        np.deg2rad(dir_section.pyramid[pli_pyramid]),
                        ret_section.pyramid[pli_pyramid][:],
                        mask_pyramid - pli_pyramid)

    mask_section = pli.data.Section(r.file_mask, data_group=mask_group)
    test_mask = mask_section.pyramid[mask_pyramid][:]

    test_valid = test_mask != background_class
    test_std = scaler.transform(test_features.reshape(-1, test_features.shape[-1]))

    test_pca = pca_n_transform(test_std, pca, n_components)
    test_pca = test_pca.reshape(*test_features.shape[:2], test_pca.shape[-1])

    out_file = os.path.basename(r.file_ret).replace('Retardation.nii', 'PCA')
    with h5.File(os.path.join(out_path, out_file), 'w') as f:
        pca_dset = f.create_dataset(pca_key, data=test_pca.transpose(2, 0, 1), dtype=np.float32)
        pca_dset.attrs['spacing'] = r.spacing
        pca_dset.attrs['origin'] = r.origin
        pca_dset.attrs['variance_explained'] = variance_explained
        pca_dset.attrs['eigenvalues'] = pca.explained_variance_
        pca_dset.attrs['n_components'] = n_components
        pca_dset.attrs['pl_components'] = pl_components

        valid_dset = f.create_dataset(valid_key, data=test_valid)
        valid_dset.attrs['spacing'] = r.spacing
        valid_dset.attrs['origin'] = r.origin

 31%|███████████████████████████████▏                                                                      | 71/232 [07:57<18:02,  6.72s/it]


KeyboardInterrupt: 

In [ ]:
test_ids = [860, 898, 961, 1061]

###

out_path = os.path.join(out_folder, 'pli', 'overview')

for test_id in test_ids:
    test_file = files_df.loc[files_df.id == test_id].iloc[0]

    trans_section = pli.data.Section(test_file.file_trans)
    dir_section = pli.data.Section(test_file.file_dir)
    ret_section = pli.data.Section(test_file.file_ret)

    test_features = pli2feat(trans_section.pyramid[pli_pyramid][:],
                             np.deg2rad(dir_section.pyramid[pli_pyramid]),
                             ret_section.pyramid[pli_pyramid][:],
                             mask_pyramid - pli_pyramid)

    mask_section = pli.data.Section(test_file.file_mask, data_group=mask_group)
    test_mask = mask_section.pyramid[mask_pyramid][:]

    test_features = np.rot90(test_features, 3)
    test_mask = np.rot90(test_mask, 3)

    test_tissue = test_mask != background_class

    test_valid = test_tissue
    test_std = scaler.transform(test_features.reshape(-1, test_features.shape[-1]))
    test_pca = pca.transform(test_std)
    test_pca = test_pca.reshape(*test_features.shape[:2], test_pca.shape[-1])

    print_pca = np.empty((*test_features.shape[:2], test_pca.shape[-1]), dtype=np.float16)
    for i in range(test_pca.shape[-1]):
        print_pca[..., i][test_valid] = test_pca[..., i][test_valid]
        print_pca[..., i][~test_valid] = np.min(test_pca[..., i])

    if not os.path.exists(out_path):
        os.mkdir(out_path)

    print_components = min(n_components, 16)

    im.show(np.stack([print_pca[..., i] for i in range(print_components)]), ch='NHW', size=16, file=out_path + f"/pca_{n_components}_s{test_id:04d}")

In [ ]:
exit()